In [ ]:
import pandas as pd

In [ ]:
big=pd.read_csv('Train.csv')
big

In [ ]:
big.shape


(8523, 12)

In [ ]:
big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
big.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
big.isnull().sum()

,0
Item_Identifier,0
Item_Weight,0
Item_Fat_Content,0
Item_Visibility,0
Item_Type,0
Item_MRP,0
Outlet_Identifier,0
Outlet_Establishment_Year,0
Outlet_Size,2410
Outlet_Location_Type,0


In [ ]:
# Handling missimg values for 'Item_Weight' (fill missing values with the mean)
big['Item_Weight'].fillna(big['Item_Weight'].mean(),inplace=True)
# Handling missing values for 'Outlet_Size' (fill missimg value baded on 'Outlet_Type')
big['Outlet_Size'].fillna(big.groupby('Outlet_Type')['Outlet_Size'].transform(lambda x:x.mode()[0]),inplace=True)

In [ ]:
big['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [ ]:
# Standardizing 'Item_Fat_Content' categories (fixing inconsistent labls)
big['Item_Fat_Content']=big['Item_Fat_Content'].replace({'LF':'Low Fat','low fat':'Low Fat','reg':'Regular'})
big['Item_Fat_Content'] = big['Item_Fat_Content'].replace({'Low Fat': 1, 'Regular': 0})


In [ ]:
# Create a new column for the age of thr store('Outlet_Age')
big['Outlet_Age']=2024-big['Outlet_Establishment_Year']
# Creat a new column for 'Item_Visibility' where zero values are replaced by the mean
big['Item_Visibility'].replace(0,big['Item_Visibility'].mean(),inplace=True)


In [ ]:

# Label encoding for 'Item_Type' and 'Outlet_identifier' as they have too many unique categories
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
big['Item_Type']=le.fit_transform(big['Item_Type'])
big['Outlet_Identifier']=le.fit_transform(big['Outlet_Identifier'])
big['Item_Identifier'] = le.fit_transform(big['Item_Identifier'])



In [ ]:
# Apply standard scaling to 'Item_MRP','Item_Weight','Item_Visibility', and 'Outlet_Age'
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
big[['Item_MRP', 'Item_Weight', 'Item_Visibility', 'Outlet_Age']] = scaler.fit_transform(big[['Item_MRP', 'Item_Weight', 'Item_Visibility', 'Outlet_Age']])



In [ ]:
big = big.drop('Item_Identifier', axis=1)


In [ ]:
from sklearn.model_selection import train_test_split

# Define target variable and features
X = big.drop('Item_Outlet_Sales', axis=1)
y = big['Item_Outlet_Sales']

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error

# Building a linear regression model
model = LinearRegression()

# Model training
model.fit(X_train, y_train)

# Prediction on test data
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")



Regression Modeling Project Report for Product Sales Prediction:

1. Introduction:
The objective of this project is to build regression models for predicting product sales. The dataset used is the Bigmart Sales Dataset, which contains information about products, store features, and product sales across different stores.

2. Data Preprocessing:
Data Cleaning: After an initial review of the data, it was observed that the Item_Weight and Outlet_Size columns had missing values. Missing values in the Item_Weight column were filled using the mean, and for the Outlet_Size column, the mode (most frequent value) was used.

Standardizing Item_Fat_Content values: Some categories in this column were inconsistently labeled (such as "low fat" and "LF"). These were corrected, and then converted into numerical values (Low Fat as 1 and Regular as 0) for use in the model.

Creating new variables:

A new column Outlet_Age was created to represent the age of the store.
Additionally, zero values in the Item_Visibility column were replaced with the mean of the column.
Categorical Encoding: The columns Item_Type and Outlet_Identifier were encoded using LabelEncoder due to the large number of unique categories.

Standardization: The columns Item_MRP, Item_Weight, Item_Visibility, and Outlet_Age were standardized using StandardScaler.

3. Modeling:
Data Split: The data was split into training (80%) and testing (20%) sets. The target variable was Item_Outlet_Sales, while the remaining columns were used as features in the model.

Linear Regression Model: A linear regression model was used to predict product sales. The model was trained on the training data and then used to make predictions on the test data.

4. Model Evaluation:
The linear regression model was evaluated using the following metrics:

Mean Squared Error (MSE):
This represents the average of the squared differences between predicted and actual values. The MSE for this model was 2,022,371.29.

Root Mean Squared Error (RMSE):
RMSE is the square root of MSE, indicating how much the model's predictions deviate from the actual values on average. The RMSE was 1,421.24, indicating the average error in predictions.

R² Score:
This metric indicates how much of the variance in the data is explained by the model. The R² score for this model was 0.5794, suggesting that the model explains approximately 57.9% of the variance in the data.

5. Conclusion:
The linear regression model has shown a reasonable accuracy in predicting product sales. However, the model could be further improved with more complex techniques and better feature selection.